# *fbref.com* Football Teams Data Web Scraping

In [39]:
# ------------------
## Import llibreries
# ------------------

import builtwith
import whois
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import re
import pandas as pd

# ---------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------



# -------------------------------
# Definició funció descàrrega URL
# -------------------------------

def download(url):
    """
    download() opens and reads a URL (and raises
    possible errors)

    :param url: URL to open and read
    :return: Read URL
    """
    try:
        html = urlopen(url).read()
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The server could not be found!')
    return html

# ---------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------



# ---------------------------------------------
# Definició paràmetres a usar al llarg del codi
# ---------------------------------------------

# Definim URL web principal
url = "https://fbref.com"


# Creo llista amb el nom de les 5 lligues
leagues = ["Premier League", "Ligue 1", "Fußball-Bundesliga",
          "Serie A", "La Liga"]


# Defineixo temporades que volem estudiar
seasons = ["2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
          "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020",
          "2020-2021"]


# Defineixo llistes amb els elements presents a les taules que volem scrapejar
# per a cada taula que scrapejarem:
league_table_elements = ["rank", "squad", "games", "wins", "draws", "losses","goals_for",
            "goals_against", "goal_diff", "points", "notes"]

squad_standard_elements = ["squad", "players_used", "assists", "pens_made", "pens_att",
                           "cards_yellow", "cards_red"]

squad_goalkeeping_elements = ["squad", "shots_on_target_against", "saves", "clean_sheets"]

squad_shooting_elements = ["squad", "shots_on_target"]

squad_playing_time_elements = ["squad", "games_starts", "games_complete", "games_subs",
                               "unused_subs", "points_per_match"]


# Defineixo diccionaris on anirem guardant les dades que volem scrapejar de cada
# taula que scrapejem
league_table = {"competition": [], "season": [], "rank": [], "squad": [], "games": [],
         "wins": [], "draws": [], "losses": [],"goals_for": [], "goals_against": [],
         "goal_diff": [], "points": [], "notes": []}

squad_standards_table = {"competition": [], "season": [], "squad": [], "players_used": [],
                         "assists": [], "pens_made": [], "pens_att": [], "cards_yellow": [],
                         "cards_red": []}

squad_goalkeeping_table = {"competition": [], "season": [], "squad": [],
                           "shots_on_target_against": [], "saves": [], "clean_sheets": []}

squad_shooting_table = {"competition": [], "season": [], "squad": [], "shots_on_target": []}

squad_playing_time_table = {"competition": [], "season": [], "squad": [], "games_starts": [],
                            "games_complete": [], "games_subs": [], "unused_subs": [],
                            "points_per_match": []}



# ---------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------



#######################
#       CODI          #
#######################

# Descarreguem i parsegem pàgina principal
fbref_html = download("https://fbref.com/en/")
fbref_bs = BeautifulSoup(fbref_html, "html.parser")

# ----------------------------------------------------------------------
## Anem a la secció de competicions des de la pàgina principal
# ----------------------------------------------------------------------

# Obtinc secció competicions i en selecciono el 'href'
comps = fbref_bs.find(id = "header_comps")
comps_href = comps.a.attrs['href']

# Creo el link sencer a les competicions
comps_url = url + comps_href
comps_url

# Descarreguem i parsegem la pàgina de competicions
comps_html = download(comps_url)
comps_bs = BeautifulSoup(comps_html, "html.parser")


# ----------------------------------------------------------------------
# Seleccionem les URL's de competició de les 5 grans lligues
# ----------------------------------------------------------------------

# Creo diccionari buit on guardaré els links a les 5 grans lligues
big5_urls = {}

# Selecciono la secció on trobem la taula amb enllaços a les 5
# grans lligues europees (`find`); després ens situem en el
# contingut de la taula (`tbody`) i seleccionem cada una de les
# línies (`tr`)
big5 = comps_bs.find(id = "all_comps_club").tbody.find_all("tr")

# Per cada una de les línies (`tr`), ens quedem amb el primer contingut,
# que és on hi trobem el "header cell" (`th`)
for row in big5:
    # Per cada "header cell" (`row.contents[0]`), ens quedem amb l'únic
    # element que hi ha (`<a>`), que conté el 'href' de la competició
    for element in row.contents[0]:
        # Mirem si la lliga es troba en les que volem seleccionar i, si
        # ho està, creem el path complet de la competició
        if element.string in leagues:
            big5_urls[element.string] = url + element.attrs['href']


# ----------------------------------------------------------------------
# Extracció de dades per cada competició i temporada
# ----------------------------------------------------------------------

# Comencem recorrent cada una de les lligues
for league in leagues:

    # Extracció de URL's de les temporades per competició
    seasons_html = download(big5_urls[league])
    seasons_bs = BeautifulSoup(seasons_html, "html.parser")

    # Ens situem a la secció on trobem totes les temporades (`id = all-seasons`),
    # Després ens situem a la taula (`table`) i després seleccionem totes les files
    # (`tr`) ja que és on trobem les temporades
    all_seasons = seasons_bs.find(id = "all_seasons").table.tbody.find_all("tr") 

    # Seleccionem la capçalera de fila (`th`), on hi troberm el 'href' del link
    # i creem el link. Creem dos diccionaris on guardarem el link complet i
    # el `href` del link (ja que el necessitarem per a les taules de lliga)
    season_links = {}
    season_href = {}

    for season in all_seasons:
        # Mirem que la temporada sigui de les que volem estudiar
        if season.th.a.text in seasons:
            season_links[season.th.a.text] = url + season.th.a['href']
            season_href[season.th.a.text] = season.th.a['href']


    # Recorrem cada una de les temporades per extreure'n les dades
    for season in seasons:

        # ----------------------------------------------------------
        # Extracció dades League Table
        # ----------------------------------------------------------

        # APROFITEM LA EXTRACCIÓ DE DADES PER APUNTAR LA COMPETICIÓ
        # I TEMPORADA EN QUÈ ENS TROBEM

        # Descarrego i parsejo la URL on es troben les 5 taules de la temporada
        season_html = download(season_links[season])
        season_bs = BeautifulSoup(season_html, "html.parser")

        # Busquem l'índex de 'href' que ens permetrà elocalitzar la "id" de la
        # League Table per aquesta temporada en concret
        league_table_ix = re.findall(r"\d+", season_href[season])[1]

        # Creem el `id` de la taula amb estadístiques generals
        league_table_id = "results" + league_table_ix + "1_overall"

        # Busquem la taula amb la nostra `id` i recorrem cada fila de
        # la taula (`tr`).
        for tr in season_bs.find(id = league_table_id).tbody.find_all("tr"):
            # Per cada element de la fila, ens quedarem amb els atributs que volem guardar
            # i anotarem competició i temporada
            for element in tr:
                if element.attrs['data-stat'] in league_table_elements:
                    if element.attrs['data-stat'] == "squad":
                        # Introduim, amb la comanda d'equip, competició i temporada
                        league_table[element.attrs['data-stat']].append(element.a.text)
                        league_table["competition"].append(league)
                        league_table["season"].append(season)
                    else:
                        league_table[element.attrs['data-stat']].append(element.string)


        # ----------------------------------------------------------
        # Extracció dades Squad Stanndard Table
        # ----------------------------------------------------------

        # APROFITEM LA EXTRACCIÓ DE DADES PER APUNTAR LA COMPETICIÓ
        # I TEMPORADA EN QUÈ ENS TROBEM

        # Busquem la taula amb la nostra `id` i recorrem cada fila de
        # la taula (`tr`).
        for tr in season_bs.find(id = "all_stats_squads_standard").tbody.find_all("tr"):
            # Per cada element de la fila, ens quedarem amb els atributs que volem guardar
            # i anotarem competició i temporada
            for element in tr:
                if element.attrs['data-stat'] in squad_standard_elements:
                    if element.attrs['data-stat'] == "squad":
                        squad_standards_table[element.attrs['data-stat']].append(element.a.text)
                        squad_standards_table["competition"].append(league)
                        squad_standards_table["season"].append(season)
                    else:
                        squad_standards_table[element.attrs['data-stat']].append(element.string)


        # ----------------------------------------------------------
        # Extracció dades Squad Goalkeeping Table
        # ----------------------------------------------------------

        # APROFITEM LA EXTRACCIÓ DE DADES PER APUNTAR LA COMPETICIÓ
        # I TEMPORADA EN QUÈ ENS TROBEM

        # Busquem la taula amb la nostra `id` i recorrem cada fila de
        # la taula (`tr`).
        for tr in season_bs.find(id = "all_stats_squads_keeper").tbody.find_all("tr"):
            # Per cada element de la fila, ens quedarem amb els atributs que volem guardar
            # i anotarem competició i temporada
            for element in tr:
                if element.attrs['data-stat'] in squad_goalkeeping_elements:
                    if element.attrs['data-stat'] == "squad":
                        squad_goalkeeping_table[element.attrs['data-stat']].append(element.a.text)
                        squad_goalkeeping_table["competition"].append(league)
                        squad_goalkeeping_table["season"].append(season)
                    else:
                        squad_goalkeeping_table[element.attrs['data-stat']].append(element.string)


        # ----------------------------------------------------------
        # Extracció dades Squad Shooting Table
        # ----------------------------------------------------------

        # APROFITEM LA EXTRACCIÓ DE DADES PER APUNTAR LA COMPETICIÓ
        # I TEMPORADA EN QUÈ ENS TROBEM

        # Busquem la taula amb la nostra `id` i recorrem cada fila de
        # la taula (`tr`).
        for tr in season_bs.find(id = "all_stats_squads_shooting").tbody.find_all("tr"):
            # Per cada element de la fila, ens quedarem amb els atributs que volem guardar
            # i anotarem competició i temporada
            for element in tr:
                if element.attrs['data-stat'] in squad_shooting_elements:
                    if element.attrs['data-stat'] == "squad":
                        squad_shooting_table[element.attrs['data-stat']].append(element.a.text)
                        squad_shooting_table["competition"].append(league)
                        squad_shooting_table["season"].append(season)
                    else:
                        squad_shooting_table[element.attrs['data-stat']].append(element.string)


        # ----------------------------------------------------------
        # Extracció dades Squad Playing Time Table
        # ----------------------------------------------------------

        # APROFITEM LA EXTRACCIÓ DE DADES PER APUNTAR LA COMPETICIÓ
        # I TEMPORADA EN QUÈ ENS TROBEM

        # Busquem la taula amb la nostra `id` i recorrem cada fila de
        # la taula (`tr`).
        for tr in season_bs.find(id = "all_stats_squads_playing_time").tbody.find_all("tr"):
            # Per cada element de la fila, ens quedarem amb els atributs que volem guardar
            for element in tr:
                if element.attrs['data-stat'] in squad_playing_time_elements:
                    if element.attrs['data-stat'] == "squad":
                        squad_playing_time_table[element.attrs['data-stat']].append(element.a.text)
                        squad_playing_time_table["competition"].append(league)
                        squad_playing_time_table["season"].append(season)
                    else:
                        squad_playing_time_table[element.attrs['data-stat']].append(element.string)


# ----------------------------------------------------------------------
# Creació DF final
# ----------------------------------------------------------------------

# Agafo totes les llistes i les convewrteixo a DF
df1 = pd.DataFrame.from_dict(league_table)
df2 = pd.DataFrame.from_dict(squad_standards_table)
df3 = pd.DataFrame.from_dict(squad_goalkeeping_table)
df4 = pd.DataFrame.from_dict(squad_shooting_table)
df5 = pd.DataFrame.from_dict(squad_playing_time_table)

# Faig un `merge` de les taules en base a competició, temporada i equip
football_teams_data = df1.merge(df2,
          how = "left",
          left_on = ["competition", "season", "squad"],
          right_on = ["competition", "season", "squad"]).merge(df3,
          how = "left",
          left_on = ["competition", "season", "squad"],
          right_on = ["competition", "season", "squad"]).merge(df4,
          how = "left",
          left_on = ["competition", "season", "squad"],
          right_on = ["competition", "season", "squad"]).merge(df5,
          how = "left",
          left_on = ["competition", "season", "squad"],
          right_on = ["competition", "season", "squad"])

# Exportem el DF final en formats 'csv' i 'xlsx'
football_teams_data.to_csv("football_teams_data.csv", index = False, encoding = "UTF-8")
football_teams_data.to_excel("football_teams_data.xlsx", index = False)